In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import os
import sys
from io import BytesIO
from math import erf, sqrt

In [ ]:
 def alerts_panel(self, df, current_period, prior_period, rel_thresh=0.5, abs_thresh=0.05, alpha=0.05):
        st.subheader("⚠️ Automated Alerts - Scrap Rate Increases")
        # Merge per-part metrics
        curr = current_period['per_part'].rename(columns={'total_defects': 'total_curr', 'scrap_count': 'scrap_curr'})
        prior = prior_period['per_part'].rename(columns={'total_defects': 'total_prior', 'scrap_count': 'scrap_prior'})
        merged = curr.merge(prior, how='left', on='part_number').fillna(0)

        # compute rates and deltas
        merged['rate_curr'] = merged.apply(lambda r: (r['scrap_curr'] / r['total_curr']) if r['total_curr'] > 0 else 0, axis=1)
        merged['rate_prior'] = merged.apply(lambda r: (r['scrap_prior'] / r['total_prior']) if r['total_prior'] > 0 else 0, axis=1)
        merged['abs_delta'] = merged['rate_curr'] - merged['rate_prior']
        merged['rel_delta'] = merged.apply(lambda r: (r['rate_curr'] / r['rate_prior'] - 1) if r['rate_prior'] > 0 else float('inf') if r['rate_curr']>0 else 0, axis=1)

        alerts = []
        for _, row in merged.iterrows():
            # significance test for proportions
            z, p = two_prop_z_test(int(row['scrap_curr']), int(row['total_curr']), int(row['scrap_prior']), int(row['total_prior']))
            signif = (p is not None and p < alpha)
            triggered = (row['rel_delta'] >= rel_thresh) or (row['abs_delta'] >= abs_thresh)
            if triggered:
                alerts.append({
                    'part_number': row['part_number'],
                    'total_curr': int(row['total_curr']),
                    'scrap_curr': int(row['scrap_curr']),
                    'rate_curr_pct': round(row['rate_curr'] * 100, 2),
                    'total_prior': int(row['total_prior']),
                    'scrap_prior': int(row['scrap_prior']),
                    'rate_prior_pct': round(row['rate_prior'] * 100, 2),
                    'abs_delta_pp': round(row['abs_delta'] * 100, 2),
                    'rel_delta_pct': round(row['rel_delta'] * 100 if row['rel_delta'] != float('inf') else 9999, 1),
                    'z': round(z, 3) if z is not None else None,
                    'p_value': round(p, 4) if p is not None else None,
                    'significant': signif
                })

        alerts_df = pd.DataFrame(alerts).sort_values(['abs_delta_pp'], ascending=False)
        if alerts_df.empty:
            st.success("No alerts triggered for the selected thresholds and period")
        else:
            st.info(f"Showing alerts where p < {alpha}, relative ≥ {rel_thresh*100}% or absolute ≥ {abs_thresh*100} pp.")
            st.dataframe(alerts_df, use_container_width=True)
            csv = alerts_df.to_csv(index=False)
            st.download_button('Download alerts CSV', csv, file_name='alerts.csv')

    def part_leaderboard(self, summary_df, top_n=15):
        st.subheader("Top Parts by Defect Count")
        if summary_df.empty:
            st.info("No part-level summary available")
            return
        display = summary_df[['part_number', 'total_defects', 'SCRAP', 'REPAIRED', 'scrap_rate_percent', 'top_reasons']].head(top_n)
        display = display.rename(columns={'SCRAP': 'scrap_count', 'REPAIRED': 'repaired_count'})
        st.dataframe(display, use_container_width=True)
        csv = display.to_csv(index=False)
        st.download_button("Download top parts CSV", csv, file_name="top_parts.csv")

    def time_trends(self, df):
        st.subheader("📊 Defect Trends Over Time")
        
        if df.empty:
            st.info("No data available for trend analysis.")
            return
        
        # Data validation and cleaning
        df = df.copy()
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=['date'])
        
        if df.empty:
            st.warning("No valid date data available after cleaning.")
            return
        
        # Simple date range filter only
        min_date, max_date = df['date'].min(), df['date'].max()
        date_range = st.date_input(
            "Select Date Range",
            value=(min_date, max_date),
            min_value=min_date,
            max_value=max_date
        )
        
        if len(date_range) == 2:
            start_date, end_date = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])
            df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
        
        if df.empty:
            st.warning("No data in selected date range.")
            return
        
        # --- DAILY DEFECT TREND ---
        daily_trend = df.groupby(df['date'].dt.date).size().reset_index()
        daily_trend.columns = ['date', 'defect_count']
        
        fig_daily = go.Figure()
        fig_daily.add_trace(go.Scatter(
            x=daily_trend['date'],
            y=daily_trend['defect_count'],
            mode='lines+markers',
            name='Daily Defects',
            line=dict(color='#1f77b4', width=3),
            marker=dict(size=6, color='#1f77b4'),
            hovertemplate='<b>%{x}</b><br>Defects: %{y}<extra></extra>'
        ))
        
        # Add 7-day moving average for better trend visibility
        if len(daily_trend) >= 7:
            daily_trend['moving_avg'] = daily_trend['defect_count'].rolling(window=7, min_periods=1).mean()
            fig_daily.add_trace(go.Scatter(
                x=daily_trend['date'],
                y=daily_trend['moving_avg'],
                mode='lines',
                name='7-Day Average',
                line=dict(color='#ff7f0e', width=2, dash='dash'),
                hovertemplate='<b>%{x}</b><br>7-Day Avg: %{y:.1f}<extra></extra>'
            ))
        
        fig_daily.update_layout(
            title="📈 Daily Defect Trend",
            template='plotly_white',
            xaxis_title="Date",
            yaxis_title="Defect Count",
            hovermode='x unified',
            showlegend=True,
            height=400
        )
        st.plotly_chart(fig_daily, use_container_width=True)
        
        # --- DISPOSITION TREND ---
        if 'disposition_norm' in df.columns:
            disposition_trend = df.groupby([df['date'].dt.date, 'disposition_norm']).size().reset_index()
            disposition_trend.columns = ['date', 'disposition', 'count']
            
            # Only show top 3 dispositions to avoid clutter
            top_dispositions = disposition_trend.groupby('disposition')['count'].sum().nlargest(3).index
            disposition_trend = disposition_trend[disposition_trend['disposition'].isin(top_dispositions)]
            
            fig_disp = px.area(
                disposition_trend,
                x='date',
                y='count',
                color='disposition',
                title="🔄 Disposition Trend Over Time",
                color_discrete_sequence=px.colors.qualitative.Set2
            )
            
            fig_disp.update_layout(
                template='plotly_white',
                xaxis_title="Date",
                yaxis_title="Count",
                hovermode='x unified',
                height=400
            )
            st.plotly_chart(fig_disp, use_container_width=True)
        
        # --- SIMPLE SUMMARY STATS ---
        st.markdown("### 📋 Trend Summary")
        col1, col2, col3, col4 = st.columns(4)
        
        with col1:
            avg_daily = daily_trend['defect_count'].mean()
            st.metric("Avg Daily Defects", f"{avg_daily:.1f}")
        
        with col2:
            max_daily = daily_trend['defect_count'].max()
            st.metric("Peak Daily Defects", f"{max_daily:.0f}")
        
        with col3:
            total_defects = daily_trend['defect_count'].sum()
            st.metric("Total Defects", f"{total_defects:,}")
        
        with col4:
            trend_direction = "📈 Increasing" if len(daily_trend) > 1 and daily_trend['defect_count'].iloc[-1] > daily_trend['defect_count'].iloc[0] else "📉 Decreasing"
            st.metric("Overall Trend", trend_direction)





    def part_detail_with_excel(self,df):
        st.subheader("📊 Part Detail & Excel Report")

        part_number = st.selectbox("Select Part Number", df["part_number"].unique())

        p_df = df[df["part_number"] == part_number]
        if p_df.empty:
            st.warning("No records for this part.")
            return

        # Excel export
        buffer = BytesIO()
        with pd.ExcelWriter(buffer, engine="xlsxwriter") as writer:
            p_df.sort_values("date", ascending=False).to_excel(writer, sheet_name="records", index=False)
            summary = pd.DataFrame({
                "metric": ["total_records", "scrap_count", "repaired_count"],
                "value": [
                    len(p_df),
                    (p_df["disposition_norm"] == "SCRAP").sum(),
                    (p_df["disposition_norm"] == "REPAIRED").sum(),
                ],
            })
            summary.to_excel(writer, sheet_name="summary", index=False)
        buffer.seek(0)

        st.download_button(
            label=f"📥 Download {part_number} Report",
            data=buffer,
            file_name=f"{part_number}_report.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )
